In [2]:
%load_ext autoreload
%autoreload 2

from netty.netty import Netty
import img_utils as im
import numpy as np
from netty import netty_utils as nutil
from IPython.display import clear_output

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
net = Netty()
net.clear()
net.args["style_layers"] = [1,4,7]
net.build()

/usr/local/lib/python3.5/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
a = im.load("data/tex/flame_braini075.jpg")
b = im.load("data/tex/exp_2.jpg")
b2 = im.load("data/tex/exp_1.jpg")
c = im.load("data/tex/br_5.jpg")
d = im.histmatch(255-c,b)
e = im.histmatch(c,b)


xs = [im.load("data/x/bok_1.jpg"), im.load("data/x/bok_2.jpg"), im.load("data/x/bok_4.jpg")]

imgs_ = [[a,b],[a],[d,b2]]
masks = None
scales_ = [
    [im.propscale([4134+100,2480+100],a.shape[:2]),
          im.propscale([4134+100,2480+100],b.shape[:2])],
          [im.propscale([4134+100,2480+100],a.shape[:2])],
    [im.propscale([4134+100,2480+100],d.shape[:2])*0.75,
          im.propscale([4134+100,2480+100],b.shape[:2])]
         ]
w = None

In [10]:
fn(xs[0],imgs_[0],scales_[0])
fn(xs[1],imgs_[1],scales_[1])
fn(xs[2],imgs_[2],scales_[2])

In [11]:
def fn(x,imgs,scales):
    render_iters = 10
    render_maxfun = 20

    hh = 2480+100
    ww = 4134+100
    w2 = ww // 2
    ovr = 600
    ov2 = ovr // 2

    x = im.size(x,(ww,hh))
    x = np.array(x)

    net.args["size"] = [w2+ov2,hh]
    net.args["maxfun"] = render_maxfun
    patches = [[[0,hh],[0,w2+ov2]],[[0,hh],[w2-ov2,ww]]]

    for j in range(render_iters):
        for i,p in enumerate(patches):
            print(p[0][0],p[0][1],p[1][0],p[1][1])

            net.set_style(imgs,masks,scales,w)
            net.set_x0(x[p[0][0]:p[0][1],p[1][0]:p[1][1]])
            print("Setup...")
            net.setup()
            print("Render!")
            x1=net.render()
            clear_output()
        #     
            if i == 0:
                x[p[0][0]:p[0][1],p[1][0]:p[1][1]-ovr] = x1[:,:-ovr]
                for y in range(ovr):
                    x[p[0][0]:p[0][1],p[1][1]-ovr+y] = x[p[0][0]:p[0][1],p[1][1]-ovr+y]*(y/ovr) + x1[:,-ovr+y]*((ovr-y-1)/ovr)
            else:
                x[p[0][0]:p[0][1],p[1][0]+ovr:p[1][1]] = x1[:,ovr:]
                for y in range(ovr):
                    x[p[0][0]:p[0][1],p[1][0]+y] = x[p[0][0]:p[0][1],p[1][0]+y]*(y/ovr) + x1[:,y]*((ovr-y-1)/ovr)

    im.save_frame(x,"data/bin",q=95)

In [27]:
x = np.array(x)
x[p[0][0]:p[0][1],p[1][0]:p[1][1]-ovr] = x1[:,:-ovr]

In [25]:
print(patches[0][0][0],patches[0][0][1],patches[0][1][0],patches[0][1][1]-ovr)
print(x[patches[0][0][0]:patches[0][0][1],patches[0][1][0]:patches[0][1][1]-ovr].shape)
print(x1[:,:-ovr].shape)
print(x1.shape)
print(x.shape)


net.args["size"]

0 2580 0 1817
(2580, 1817, 3)
(2580, 1817, 3)
(2580, 2417, 3)
(2580, 4234, 3)


[2417, 2580]